# Es necesario informar el días desde que se van a recoger datos y el número de dias, así como el código del aeropueto
Buscar aquí: https://www.wunderground.com/history

In [5]:
#Dependecias Externas

import sys
import requests
from bs4 import BeautifulSoup
import xlsxwriter
import pandas as pd
import time
import datetime
from metar import Metar


In [6]:
#Definición de Funciones

def lista_fecha(anio, mes, dia, dias):
    datelist = pd.date_range(datetime.datetime(anio, mes, dia), periods=dias).tolist()
    result = map(lambda x: x.to_pydatetime().date().strftime('%Y/%m/%d'), datelist)
    return list(result)

def leer_meteo(url):
    headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5)",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "accept-charset": "ISO-8859-1,utf-8;q=0.7,*;q=0.3",
    "accept-encoding": "gzip,deflate,sdch",
    "accept-language": "en-US,en;q=0.8",
    }

    #Metemos las cookies para que el html que devuelva sea el que necesitamos
    cook = dict(Prefs='FAVS:1|WXSN:1|PWSOBS:1|WPHO:1|PHOT:1|RADC:0|RADALL:0|HIST0:NULL|GIFT:1|SHOWMETAR:1|')
    
    r = requests.get(url, headers=headers, cookies=cook)
    
    if r.status_code != 200:
        print("request denied")
        print(r.status_code)
        sigue = 0

    soup = BeautifulSoup(r.text, 'html.parser')
    lista_info = []
    try:
        table =  soup.select("#obsTable")[0]
        for tr in table.find_all('tr'):
            for tds in tr.find_all('td', attrs={"colspan":"12"}):
                lista_info.append(tds.text)
                #print(tds.text)
        return lista_info
    except:
        pass

def filtrar_metars (lista):
    result = []
    try:
        lista.sort()
    
        for linea in lista:
            if linea[:5] == 'METAR':
                result.append(linea)
        return result
    except:
        pass

def metar_parser(solo_metars, ubicacion, fecha):
    metar = []
    try:
        for element in solo_metars:
            lluvia = 0
            dir_viento = 0
            fuerza_viento = 0
            temp = 0
            
            # Initialize a Metar object with the data from solo_metars
            try:
                obs = Metar(element)
            except:
                pass
            
            estacion = ubicacion

            try:
                h = "00"+str(obs.time.hour)
                m = "00"+str(obs.time.minute)
                hora = fecha+" "+h+":"+m[-2:]
            except:
                hora = ''

            try:
                temp = obs.temp.value('C')                  
            except:
                temp = ''

            try:
                if obs.wind_dir.value()==360.0:
                    dir_viento = 0
                else:
                    dir_viento = obs.wind_dir.value()
            except:
                dir_viento = ''

            try:
                fuerza_viento = obs.wind_speed.value("KMH")
            except:
                fuerza_viento = ''

            try:
                lluvia = obs.precip_1hr.string("in")/0.061024
            except:
                lluvia = ''
            
            #print(estacion, hora, temp, fuerza_viento, dir_viento, lluvia)
            metar.append([estacion, hora, temp, fuerza_viento, dir_viento, lluvia])
        return metar
    except:
        pass


# Probamos a hacer una funcion que recibiendo parámetros de Aeropuerto y fecha devuelva el DataFrame de ese día

In [7]:
def get_viento(aeropuerto,dia):
    #Formato de fecha admitida "2018/5/7"
    #print(dia)
    #print(aeropuerto)
    url = 'https://www.wunderground.com/history/airport/'+aeropuerto+'/'+dia+'/DailyHistory.html'
    #print(url)
    meteo_list = leer_meteo(url)
    solo_metars = filtrar_metars(meteo_list)
    info = metar_parser(solo_metars, aeropuerto, dia)
    dfinfo = pd.DataFrame(info)
    try:
        dfinfo.columns = ['Estacion','Fecha_Hora','Temp','Fuerza_Viento','Direccion_Viento','Lluvia']
        dfinfo['Fecha_Hora'] = pd.to_datetime(dfinfo['Fecha_Hora'])
    except:
        pass
    
    return
    
    out = dfinfo.convert_objects(convert_numeric=True)
    out = out[out["Direccion_Viento"].notnull()]
    out = out[out["Fecha_Hora"].dt.hour>=10]
    out = out[out["Fecha_Hora"].dt.hour<=20]
    out = out.groupby(['Estacion'])["Fuerza_Viento","Direccion_Viento"].mean()
    #print(out.dtypes)
    list_out=list(out.values)
    return list_out[0][0],list_out[0][1]
#Devuelve Array[Fuerza, Velocidad]

In [23]:
aeropuerto = 'LEVS'
dia = "2018/4/22"

In [24]:
url = 'https://www.wunderground.com/history/airport/'+aeropuerto+'/'+dia+'/DailyHistory.html'
print(url)

https://www.wunderground.com/history/airport/LEVS/2018/4/22/DailyHistory.html


In [25]:
meteo_list = leer_meteo(url)
solo_metars = filtrar_metars(meteo_list)
info = metar_parser(solo_metars, aeropuerto, dia)
dfinfo = pd.DataFrame(info)
try:
    dfinfo.columns = ['Estacion','Fecha_Hora','Temp','Fuerza_Viento','Direccion_Viento','Lluvia']
    dfinfo['Fecha_Hora'] = pd.to_datetime(dfinfo['Fecha_Hora'])
except:
    pass

In [26]:
dfinfo

,Estacion,Fecha_Hora,Temp,Fuerza_Viento,Direccion_Viento,Lluvia
0,LEVS,NaT,,,,
1,LEVS,NaT,,,,
2,LEVS,NaT,,,,
3,LEVS,NaT,,,,
4,LEVS,NaT,,,,
5,LEVS,NaT,,,,
6,LEVS,NaT,,,,
7,LEVS,NaT,,,,
8,LEVS,NaT,,,,
9,LEVS,NaT,,,,


In [20]:
out = dfinfo.convert_objects(convert_numeric=True)

#out = out[out["Direccion_Viento"].notnull()]
#out = out[out["Fecha_Hora"].dt.hour>=10]
#out = out[out["Fecha_Hora"].dt.hour<=20]
#out = out.groupby(['Estacion'])["Fuerza_Viento","Direccion_Viento"].mean()
#print(out.dtypes)
list_out=list(out.values)


C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [21]:
list_out

[array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=object),
 array(['LEMD', NaT, '', '', '', ''], dtype=ob

In [ ]:
url = 'https://www.wunderground.com/history/airport/'+aeropuerto+'/'+dia+'/DailyHistory.html'
#print(url)
meteo_list = leer_meteo(url)
solo_metars = filtrar_metars(meteo_list)
info = metar_parser(solo_metars, aeropuerto, dia)
dfinfo = pd.DataFrame(info)
try:
    dfinfo.columns = ['Estacion','Fecha_Hora','Temp','Fuerza_Viento','Direccion_Viento','Lluvia']
    dfinfo['Fecha_Hora'] = pd.to_datetime(dfinfo['Fecha_Hora'])
except:
    pass

return

out = dfinfo.convert_objects(convert_numeric=True)
out = out[out["Direccion_Viento"].notnull()]
out = out[out["Fecha_Hora"].dt.hour>=10]
out = out[out["Fecha_Hora"].dt.hour<=20]
out = out.groupby(['Estacion'])["Fuerza_Viento","Direccion_Viento"].mean()
#print(out.dtypes)
list_out=list(out.values)


In [9]:
test = get_viento('LEMD',"2018/5/7")
test

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


DataError: No numeric types to aggregate

In [ ]:
headers = {
"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5)",
"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
"accept-charset": "ISO-8859-1,utf-8;q=0.7,*;q=0.3",
"accept-encoding": "gzip,deflate,sdch",
"accept-language": "en-US,en;q=0.8",
}

#Metemos las cookies para que el html que devuelva sea el que necesitamos
cook = dict(Prefs='FAVS:1|WXSN:1|PWSOBS:1|WPHO:1|PHOT:1|RADC:0|RADALL:0|HIST0:NULL|GIFT:1|SHOWMETAR:1|')

r = requests.get(url, headers=headers, cookies=cook)

if r.status_code != 200:
    print("request denied")
    print(r.status_code)
    sigue = 0

soup = BeautifulSoup(r.text, 'html.parser')
lista_info = []
try:
    table =  soup.select("#obsTable")[0]
    for tr in table.find_all('tr'):
        for tds in tr.find_all('td', attrs={"colspan":"12"}):
            lista_info.append(tds.text)
            #print(tds.text)
    return lista_info
except:
    pass